# 🍁Segmenting and Clustering Neighborhoods in Toronto

# I. Part 1

## 1. Web Scraping

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
print("requests\t",requests.__version__,"\t| file: ",requests.__file__)
print("numpy\t\t",np.__version__,"\t| file: ",np.__file__)
print("pandas\t\t",pd.__version__,"\t| file: ",pd.__file__)

requests	 2.18.4 	| file:  /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/requests/__init__.py
numpy		 1.13.3 	| file:  /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/numpy/__init__.py
pandas		 0.21.0 	| file:  /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/__init__.py


### 1.1. URL

In [3]:
url_postal_canada = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resp = requests.get(url_postal_canada)

In [4]:
soup = BeautifulSoup(resp.text, 'html.parser')

### 1.2. Extract table

In [5]:
table = soup.find(class_='wikitable sortable')
table_rows = table.find_all('tr')

In [6]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n', '') for tr in td]
    l.append(row)

canada = pd.DataFrame(l, columns=['Postcode','Borough','Neighbourhood'])
canada.head(10)

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


## 2. Create the dataframe

### 2.1. Ignore cells with a Borough that is Not assigned.

In [7]:
canada = canada[~canada['Borough'].isin([None,'Not assigned'])]
canada.head(15)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### 2.2. Complete cells with a Neighbourhood that is Not assigned.

In [8]:
canada[canada['Neighbourhood'].isin(['Not assigned'])]

,Postcode,Borough,Neighbourhood
9,M7A,Queen's Park,Not assigned


In [9]:
canada['Neighbourhood'] = np.where(canada['Neighbourhood']=='Not assigned', canada['Borough'], canada['Neighbourhood'])

canada[canada['Neighbourhood'].isin(['Not assigned'])]

,Postcode,Borough,Neighbourhood


In [10]:
canada[canada['Postcode'].isin(['M7A'])]

,Postcode,Borough,Neighbourhood
9,M7A,Queen's Park,Queen's Park


In [11]:
canada.shape

(212, 3)

### 2.3. Combine row with same 'Postcode' & 'Borough' into one row with the 'Neighborhoods' separated with a comma

In [12]:
canada = canada.groupby(['Postcode', 'Borough'], as_index=False).agg({'Neighbourhood':', '.join})
canada.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### 2.4. Number of rows of dataframe

In [13]:
print('Number of rows of dataframe is: ',canada.shape[0])

Number of rows of dataframe is:  103


# II. Part 2

In [14]:
#! conda install -c conda-forge geocoder

In [15]:
import geocoder # import geocoder

In [16]:
def get_lat_lon(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return(latitude, longitude)

In [17]:
#canada['Latitude'], canada['Longitude'] = get_lat_lon(canada['Postcode'])
#canada.head()

In [18]:
geospatial_data = pd.read_csv('http://cocl.us/Geospatial_data')
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
geospatial_data.rename(columns={'Postal Code':'PostalCode'}, 
                       inplace=True)
geospatial_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
canada.rename(columns={'Postcode':'PostalCode'},
              inplace=True)
canada.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
canada_ll = pd.merge(canada, geospatial_data, on='PostalCode')
canada_ll.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# III. Part 3

Work with only boroughs that contain the word Toronto:

In [22]:
toronto = canada_ll.loc[canada_ll['Borough'].str.contains("Toronto")]
toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [23]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [24]:
# create map
latitude= 43.70011
longitude=-79.4163

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
map_toronto

In [25]:
neigh = folium.map.FeatureGroup()

for lat, lng, in zip(toronto.Latitude, toronto.Longitude):
    neigh.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
map_toronto.add_child(neigh)